In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('final_file.csv',encoding="GBK").drop(['Unnamed: 0'], axis=1)
X = df.values[:, 0:216]
Y = df.values[:,216]

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [3]:
df.head()

,shop_id,date.day_1,date.day_2,date.day_3,date.day_4,date.day_5,date.day_6,date.day_7,date.day_8,date.day_9,...,cate_2_name_快餐,cate_2_name_本地购物,cate_2_name_汤/粥/煲/砂锅/炖菜,cate_2_name_火锅,cate_2_name_烘焙糕点,cate_2_name_烧烤,cate_2_name_网吧网咖,cate_2_name_药店,cate_2_name_超市,count
0,544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221
1,810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125
2,163,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,91
3,1977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137
4,746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68


In [4]:
from sklearn import datasets, linear_model, cross_validation, grid_search
from sklearn.metrics import r2_score 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
import sklearn
sklearn.__version__

'0.18.1'

In [7]:
r2_results = []
mse_results= []

kf_total = KFold(n_splits=2, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X):
    x_tr, x_cv = X[train_index], X[test_index]
    y_tr, y_cv = Y[train_index], Y[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params = {"objective": "reg:linear", "booster":"gbtree","max_depth":60}
    gbm = xgb.train(dtrain=T_train_xgb,params=params)
    y_pr = gbm.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
    
print np.average(r2_results)
print np.average(mse_results)

0.851709436989
2631.5385578


In [10]:
y_pred = gbm.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.987571751453
225.28010213


In [ ]:
params2 = {"objective": "reg:linear", "booster":"gbtree","max_depth":60}
gbm2 = xgb.train(dtrain=T_train_xgb,params=params2)
y_pred2 = gbm2.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred2))
print(mean_squared_error(y_test, y_pred2))

In [12]:
df1 = df[df['shop_id'] != 1629]
df1 = df[df['shop_id'] != 58]
df1 = df[df['shop_id'] != 1821]
#df['shop_id'] != 58 and df['shop_id'] != 1821]

In [15]:
X = df1.values[:, 0:216]
Y = df1.values[:,216]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [ ]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=50, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X):
    x_tr, x_cv = X[train_index], X[test_index]
    y_tr, y_cv = Y[train_index], Y[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params_1 = {"booster":"gbtree", "objective": "reg:linear", "eta":0.1, "max_depth":40,"eval_metric":"rmse"}
    gbm = xgb.train(dtrain=T_train_xgb,params=params)
    y_pr = gbm.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)

In [ ]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=50, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X):
    x_tr, x_cv = X[train_index], X[test_index]
    y_tr, y_cv = Y[train_index], Y[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    params_1 = {"booster":"gbtree", "objective": "reg:linear", "eta":0.1, "max_depth":40,"eval_metric":"rmse"}
    gbm_12 = xgb.train(dtrain=T_train_xgb,params=params1)
    y_pr = gbm_12.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)